In [ ]:
import pandas as pd
df = pd.read_csv("../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
df["Category"] = encoder.fit_transform(df["Category"])

In [ ]:
y = df["Category"]
X = df["Message"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 69)

In [ ]:
import re

def normalize(data):
    normalized = []
    for i in data:
        i = i.lower()
        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', '', i)
        # get rid of non words and extra spaces
        i = re.sub('\\W', ' ', i)
        i = re.sub('(?<=[a-z])\'(?=[a-z])', '', i)
        i = re.sub("(?<=[a-z])'(?=[a-z])", "", i)
        i = re.sub('\n', '', i)
        i = re.sub(' +', ' ', i)
        i = re.sub('^ ', '', i)
        i = re.sub(' $', '', i)
        normalized.append(i)
    return normalized

In [ ]:
X_train = normalize(X_train)
X_test = normalize(X_test)

In [ ]:
from keras.preprocessing.text import Tokenizer

max_vocab = 10000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(X_train)

In [ ]:
# tokenize the text into vectors 
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=256)
X_test = pad_sequences(X_test, padding='post', maxlen=256)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM

model = Sequential()

In [ ]:
model.add(Embedding(max_vocab, 32))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Bidirectional(LSTM(16)))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1))

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = "accuracy")

In [ ]:
model.fit(X_train, y_train, epochs = 10, validation_split = 0.2, batch_size = 30, shuffle = True)